In [13]:
pip install beautifultable

In [14]:
import requests
import string
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from unicodedata import numeric
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import re
import random
import pandas as pd
#https://stackoverflow.com/questions/28639677/capitalize-the-first-letter-after-a-punctuation/28639714
from beautifultable import BeautifulTable

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
URL = "https://www.allrecipes.com/recipe/11679/homemade-mac-and-cheese/"
URL_nonveg = 'https://www.allrecipes.com/recipe/258947/mushroom-beef-burgers/'
URL_unhealthy = 'https://www.allrecipes.com/recipe/8338/cannoli-with-chocolate-chips/'
main_path = "/content/drive/MyDrive/NLP2/"
non_veg_file_path = main_path + "list_of_non_veg_items.txt"
veg_subs_path = main_path + "list_of_veg_substitutes.txt"
tools_path = main_path + "list_of_tools.txt"
cooking_method_path = main_path + "list_of_cooking_methods.txt"
healthy_ingredients_subs_path = main_path + "healthy_ingredients_subs.csv"
healthy_steps_subs_path = main_path + "healthy_steps.csv"

In [16]:
class Ingredient:

  def __init__(self,fullname=None,main_ingredients=None,quantity_type=None,quantity = None,units=None,descriptor=None,preparation=None,step_indexes=[]):  
    self.fullname = fullname
    self.main_ingredients = main_ingredients
    self.quantity_type = quantity_type
    self.quantity = quantity
    self.units = units
    self.descriptor = descriptor
    self.preparation = preparation
    self.step_indexes = step_indexes

  def print_ingredient(self):
    print('Full Name: ',self.fullname)
    print("Main Ingredient: ",self.main_ingredients)
    print("Quantity Type: ",self.quantity_type)
    print("Quantity: ",self.quantity)
    print("Units: ",self.units)
    print('Descriptor: ',self.descriptor)
    print('Preparation: ',self.preparation)
    print('Used in steps',self.step_indexes,'\n')

In [17]:
class Step:
  def __init__(self,full_step=None,ingredients=[],tools=[],methods = [],times=None,index=None):  
    self.full_step = full_step
    self.ingredients = ingredients
    self.tools = tools
    self.methods = methods
    self.times = times
    self.index = index
  
  def print_steps(self):
    print('Step number: ',self.index)
    print('Full Step: ',self.full_step)
    print("Ingredients: ",end='')
    for ingredient in self.ingredients:
      print(ingredient.fullname,end=',')
    print("\nTools: ",self.tools)
    print("Methods: ",self.methods)
    print("Times: ",self.times,'\n')

In [25]:
class Recipe:
  def __init__(self,Ingredients=None,Tools=None,Methods=None,Steps=None,URL=None):  
    self.Ingredients = Ingredients
    self.Tools = Tools
    self.Methods = Methods
    self.Steps = Steps
    self.URL = URL
  
  def print_recipie(self):
    '''for ingredient in self.Ingredients:
      ingredient.print_ingredient()
    
    for step in self.Steps:
      step.print_steps()
    '''
    self.print_table_ingredients()
    self.print_table_steps()

  
  def print_table_ingredients(self):
    table = BeautifulTable(maxwidth=130)
    table.columns.header = ["Full Name", "Main Ingredient", "Quantity Type","Quantity","Units","Descriptor","Preperation","Used in steps"]
    for ingredient in self.Ingredients:
      table.rows.append([ingredient.fullname,ingredient.main_ingredients,ingredient.quantity_type,ingredient.quantity,ingredient.units,ingredient.descriptor,ingredient.preparation,ingredient.step_indexes])
    print(table)
  
  def print_table_steps(self):
    table = BeautifulTable(maxwidth=130)
    table.columns.header = ["Step number", "Full Step", "Ingredients","Tools","Methods","Times"]
    for step in self.Steps:
      step_ingredients = []
      for ingredient in step.ingredients:
        step_ingredients.append(ingredient.fullname)
      table.rows.append([step.index,step.full_step,step_ingredients,step.tools,step.methods,step.times])
    print(table)

  def substitute_ingredient_fn(self,old_ingredient, substitute_ingredient):
    
    index_of_ingredient = 0

    for i in range(len(self.Ingredients)):
      if old_ingredient.lower() in self.Ingredients[i].fullname.lower() or old_ingredient.lower() in self.Ingredients[i].main_ingredients.lower():
        self.Ingredients[i].fullname = substitute_ingredient
        self.Ingredients[i].main_ingredients = substitute_ingredient
        index_of_ingredient = i
        break
       
    for i in self.Ingredients[index_of_ingredient].step_indexes:
      self.Steps[i].full_step = re.sub(old_ingredient, substitute_ingredient, self.Steps[i].full_step, flags=re.IGNORECASE) 
      #self.Steps[i].full_step = self.Steps[i].full_step.replace(old_ingredient,substitute_ingredient)
      for j in range(len(self.Steps[i].ingredients)):
        if old_ingredient.lower() in self.Steps[i].ingredients[j].fullname.lower() or old_ingredient.lower() in self.Steps[i].ingredients[j].main_ingredients.lower():
          self.Steps[i].ingredients[j] = self.Ingredients[index_of_ingredient]
          break

  def substitute_ingredient_fn(self,old_ingredient, substitute_ingredient):
    
    index_of_ingredient = 0

    for i in range(len(self.Ingredients)):
      if old_ingredient.lower() in self.Ingredients[i].fullname.lower() or old_ingredient.lower() in self.Ingredients[i].main_ingredients.lower():
        self.Ingredients[i].fullname = substitute_ingredient
        self.Ingredients[i].main_ingredients = substitute_ingredient
        index_of_ingredient = i
        break
       
    for i in self.Ingredients[index_of_ingredient].step_indexes:
      self.Steps[i].full_step = re.sub(old_ingredient, substitute_ingredient, self.Steps[i].full_step, flags=re.IGNORECASE) 
      #self.Steps[i].full_step = self.Steps[i].full_step.replace(old_ingredient,substitute_ingredient)
      for j in range(len(self.Steps[i].ingredients)):
        if old_ingredient.lower() in self.Steps[i].ingredients[j].fullname.lower() or old_ingredient.lower() in self.Steps[i].ingredients[j].main_ingredients.lower():
          self.Steps[i].ingredients[j] = self.Ingredients[index_of_ingredient]
          break

  def substitute_step_fn(self,old_step, new_step):
    
    index_of_ingredient = 0

    for i in range(len(self.Methods)):
      if old_step.lower() in self.Methods[i].lower():
        self.Methods[i] = new_step
    
    for i in range(len(self.Steps)):
      if old_step.lower() in self.Steps[i].full_step.lower() or old_step.lower() in self.Steps[i].full_step.lower():
        self.Steps[i].full_step = re.sub(old_step, new_step, self.Steps[i].full_step, flags=re.IGNORECASE) 
        for j in range(len(self.Steps[i].methods)):
          if old_step.lower() in self.Steps[i].methods[j]:
            self.Steps[i].methods[j] = new_step
        
  def to_veg(self):
    with open(non_veg_file_path) as file:
      lines = file.readlines()
      list_of_non_veg_items = [line.rstrip() for line in lines]

    with open(veg_subs_path) as file:
      lines = file.readlines()
      list_of_veg_substitutes = [line.rstrip() for line in lines]
  
    old_ingredient = None
    for i in range(len(self.Ingredients)):
      for non_veg_item in list_of_non_veg_items:
        if non_veg_item in self.Ingredients[i].fullname.lower() or non_veg_item in self.Ingredients[i].main_ingredients.lower():
         old_ingredient = non_veg_item

    if (old_ingredient):
      substitute_ingredient = random.choice(list_of_veg_substitutes)
      self.substitute_ingredient_fn(old_ingredient, substitute_ingredient)
    
  def ratio(self,ratio_num):
    for i in range(len(self.Ingredients)):
      if self.Ingredients[i].quantity.isnumeric():
        self.Ingredients[i].quantity=float(self.Ingredients[i].quantity)*ratio_num

  def to_healthy(self):
    df_ingredients = pd.read_csv(healthy_ingredients_subs_path)
    df_ingredients = df_ingredients.to_numpy()

    df_steps = pd.read_csv(healthy_steps_subs_path)
    df_steps= df_steps.to_numpy()

    for i in range(len(self.Ingredients)):
      if 'salt' in self.Ingredients[i].fullname.lower() or 'salt' in self.Ingredients[i].main_ingredients.lower():
        self.Ingredients[i].quantity *= 0.5
      if 'butter' in self.Ingredients[i].fullname.lower() or 'butter' in self.Ingredients[i].main_ingredients.lower():
        self.Ingredients[i].quantity *= 0.75
      for unhealthy_item,substitute_ingredient in df_ingredients:
        if unhealthy_item.lower() in self.Ingredients[i].fullname.lower() or unhealthy_item.lower() in self.Ingredients[i].main_ingredients.lower():
          self.substitute_ingredient_fn(unhealthy_item, substitute_ingredient)
  
    for i in range(len(self.Steps)):
      for unhealthy_step,substitute_step in df_steps:
          if unhealthy_step in self.Steps[i].full_step.lower():
            self.substitute_step_fn(unhealthy_step, substitute_step)       
      

In [19]:
def get_recipie_from_URL(URL):
  myRecipie = Recipe()
  myRecipie.URL = URL
  page = requests.get(URL)
  soup = BeautifulSoup(page.content, "html.parser")
  ingredient_parse = soup.find_all("input", class_="checkbox-list-input")
  direction_parse = soup.find_all("div", class_="paragraph")
  unicode_quantities = []
  unicode_units = []
  unicode_ingredients = []
  unicode_type = []

  #----------------------------Half Parsing Directions-------------------------#
  directions = []

  for step in direction_parse:
    step_text = step.text
    step_text.encode("ascii", "ignore").strip()
    directions.append(step_text)

  #--------------------------Half Parsed Directions are ready------------------#

  #----------------------------Parsing Ingredients-----------------------------#
  for ingredient in ingredient_parse:
    val = ingredient.attrs
    if u'data-quantity' in val.keys():
        unicode_quantities.append(val[u'data-init-quantity'])
    if u'data-unit' in val.keys():
        unicode_units.append(val[u'data-unit'])
    if u'data-ingredient' in val.keys():
        unicode_ingredients.append(val[u'data-ingredient'])
    if u'data-ingredient' in val.keys():
        unicode_type.append(val[u'data-unit_family'])

  quantities = unicode_quantities
  units = unicode_units
  ingredients = unicode_ingredients
  quantity_type = unicode_type
  descriptor  = [None] * len(ingredients)
  preparation = [None] * len(ingredients)
  main_ingredients = [''] * len(ingredients)

  def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent
  
  for i in range(len(ingredients)):
    tokens = preprocess(ingredients[i])
    for token in tokens:
      if token[1] == 'JJ':
        descriptor[i] = token[0]
      elif token[1] in ['VBD','VBN']:
        preparation[i] = token[0]
      elif token[1] in ['NN','NNS','NNP','NNPS']:
        main_ingredients[i] += token[0]+' '
    main_ingredients[i] = main_ingredients[i].rstrip()

  Ingredients = [0] * len(ingredients)
  for i in range(len(ingredients)):
    new_ingredient = Ingredient(fullname=ingredients[i],main_ingredients=main_ingredients[i],quantity_type=quantity_type[i],quantity = quantities[i],units=units[i],descriptor=descriptor[i],preparation=preparation[i])
    Ingredients[i] = new_ingredient
  
  for i in range(len(Ingredients)):
    step_indexes = []
    for j in range(len(directions)):
      direction_lower = directions[j]
      if (Ingredients[i].main_ingredients in direction_lower or Ingredients[i].main_ingredients in direction_lower) or Ingredients[i].main_ingredients.replace(" ", "") in direction_lower:
        step_indexes.append(j)
    Ingredients[i].step_indexes = step_indexes
  
  myRecipie.Ingredients = Ingredients
  #----------------------------Ingredients are ready---------------------------#
  
  #----------------------------Parsing Tools-----------------------------------#
  with open(tools_path) as file:
      lines = file.readlines()
      list_of_tools = [line.rstrip() for line in lines]

  Tools = []
  for i in range(len(directions)):
    direction_lower = directions[i]
    for tool in list_of_tools:
      if tool in direction_lower:
        Tools.append(tool)

  myRecipie.Tools = Tools
  #----------------------------Tools are ready--------------------------#

  #----------------------------Parsing Methods--------------------------#
  Cooking_Methods = []
  with open(cooking_method_path) as file:
      lines = file.readlines()
      list_of_cooking_methods = [line.rstrip() for line in lines]
  
  lemmatizer = WordNetLemmatizer()
  for i in range(len(directions)):
    direction_lower = directions[i].lower()
    for cooking_method in list_of_cooking_methods:
      if cooking_method in direction_lower or lemmatizer.lemmatize(cooking_method) in direction_lower:
        Cooking_Methods.append(cooking_method)
        
  myRecipie.Methods = Cooking_Methods
  #----------------------------Methods are ready--------------------------#

  #----------------------------Parsing Directions--------------------------#
  Steps = []

  for i in range(len(directions)):
    direction_lower = directions[i].lower()
    step_wise_ingredients = []
    step_wise_tools = []
    step_wise_methods = []
    step_wise_times = ''
    
    for tool in Tools:
      if tool in direction_lower and tool not in step_wise_tools:
        step_wise_tools.append(tool)
    
    for method in Cooking_Methods:
      if method in direction_lower and method not in step_wise_methods:
        step_wise_methods.append(method)
    
    for ingredient in Ingredients:
      if ingredient.main_ingredients in direction_lower and ingredient.fullname not in step_wise_ingredients :
        step_wise_ingredients.append(ingredient)
    
    time_keywords = 'minutes|seconds|hours|secs|hrs|mins|min'
    if (re.search(time_keywords,direction_lower)):
      index = re.search(time_keywords,direction_lower).start()
      new_str = direction_lower[:index]
      step_wise_times+= str(new_str.split()[-1])
      unit = re.findall(time_keywords,direction_lower)[-1]
      step_wise_times += ' '+str(unit)
    new_step = Step (full_step=directions[i],ingredients=step_wise_ingredients,tools=step_wise_tools,methods = step_wise_methods,times=step_wise_times,index=i)
    Steps.append(new_step)
  myRecipie.Steps = Steps

    
  return myRecipie
 #----------------------------Directions are ready--------------------------#

In [26]:
myRecipie = get_recipie_from_URL(URL)
myRecipie.print_recipie()

+---------------------------+---------------+---------------+----------+-------------+-------------+-------------+---------------+
|         Full Name         | Main Ingredie | Quantity Type | Quantity |    Units    | Descriptor  | Preperation | Used in steps |
|                           |      nt       |               |          |             |             |             |               |
+---------------------------+---------------+---------------+----------+-------------+-------------+-------------+---------------+
|  uncooked elbow macaroni  | elbow macaron |    weight     |    8     |   ounces    |  uncooked   |             |      []       |
|                           |       i       |               |          |             |             |             |               |
+---------------------------+---------------+---------------+----------+-------------+-------------+-------------+---------------+
| shredded sharp Cheddar ch | Cheddar chees |  volumetric   |    2     |    cups   

In [27]:
myRecipie = get_recipie_from_URL(URL)
myRecipie.substitute_ingredient_fn('milk','soy milk')
myRecipie.print_recipie()

+---------------------------+---------------+---------------+----------+-------------+-------------+-------------+---------------+
|         Full Name         | Main Ingredie | Quantity Type | Quantity |    Units    | Descriptor  | Preperation | Used in steps |
|                           |      nt       |               |          |             |             |             |               |
+---------------------------+---------------+---------------+----------+-------------+-------------+-------------+---------------+
|  uncooked elbow macaroni  | elbow macaron |    weight     |    8     |   ounces    |  uncooked   |             |      []       |
|                           |       i       |               |          |             |             |             |               |
+---------------------------+---------------+---------------+----------+-------------+-------------+-------------+---------------+
| shredded sharp Cheddar ch | Cheddar chees |  volumetric   |    2     |    cups   

In [22]:
myRecipie = get_recipie_from_URL(URL_nonveg)
myRecipie.to_veg()
myRecipie.print_recipie()

+----------------------+-----------------+--------+-----------+-----------+-------+-------+--------+
|      Full Name       | Main Ingredient | Quanti | Quantity  |   Units   | Descr | Prepe | Used i |
|                      |                 | ty Typ |           |           | iptor | ratio | n step |
|                      |                 |   e    |           |           |       |   n   |   s    |
+----------------------+-----------------+--------+-----------+-----------+-------+-------+--------+
| 12 ounce can chickpe | 12 ounce can ch | weight |     2     |  pounds   |       |       |  [0]   |
|     as, drained      | ickpeas, draine |        |           |           |       |       |        |
|                      |        d        |        |           |           |       |       |        |
+----------------------+-----------------+--------+-----------+-----------+-------+-------+--------+
| mushrooms, chopped,  |    mushrooms    |  each  |     1     | (8 ounce) |       | chopp |

In [23]:
myRecipie = get_recipie_from_URL(URL)
myRecipie.ratio(2)
myRecipie.print_recipie()

+----------------------+-----------+----------+-----------+---------+---------+---------+----------+
|      Full Name       | Main Ingr | Quantity | Quantity  |  Units  | Descrip | Prepera | Used in  |
|                      |  edient   |   Type   |           |         |   tor   |  tion   |  steps   |
+----------------------+-----------+----------+-----------+---------+---------+---------+----------+
| uncooked elbow macar | elbow mac |  weight  |   16.0    | ounces  | uncooke |         |    []    |
|         oni          |   aroni   |          |           |         |    d    |         |          |
+----------------------+-----------+----------+-----------+---------+---------+---------+----------+
| shredded sharp Chedd | Cheddar c | volumetr |    4.0    |  cups   |  sharp  | shredde |    []    |
|      ar cheese       |   heese   |    ic    |           |         |         |    d    |          |
+----------------------+-----------+----------+-----------+---------+---------+---------+--

In [24]:
myRecipie = get_recipie_from_URL(URL_unhealthy)
myRecipie.to_healthy()
myRecipie.print_recipie()

+---------------------+--------------+----------+----------+--------+---------+---------+----------+
|      Full Name      | Main Ingredi | Quantity | Quantity | Units  | Descrip | Prepera | Used in  |
|                     |     ent      |   Type   |          |        |   tor   |  tion   |  steps   |
+---------------------+--------------+----------+----------+--------+---------+---------+----------+
|        honey        |    honey     | volumetr |    1     |  cup   |         |         |   [0]    |
|                     |              |    ic    |          |        |         |         |          |
+---------------------+--------------+----------+----------+--------+---------+---------+----------+
| part-skim ricotta c | ricotta chee | volumetr |    1     |  pint  | part-sk |         |   [0]    |
|        heese        |      se      |    ic    |          |        |   im    |         |          |
+---------------------+--------------+----------+----------+--------+---------+---------+--